In [130]:
!pip install -qqq torch pandas numpy pytorch_lightning sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


3907.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [131]:
# Standard libraries
import os
import csv

# Standard data processing libraries
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from pytorch_lightning import LightningDataModule, LightningModule
from torch.utils.data import random_split, DataLoader

# ML libraries
import torch
from torch.utils.data import Dataset
from pytorch_lightning import Trainer

from lib.utils import types_to_int, TYPES_MAP

In [132]:
PROJECT_DIR = "./" # @param {type: "string"}
DATASET = "headlines" # @param {type: "string"}
DATA_DIR = os.path.join(PROJECT_DIR, "data/sem_eval_2016/", DATASET)

TRAIN_BATCH_SIZE = 16 # @param {type: "slider", min:1, max:128}
BATCH_SIZE = 16 # @param {type: "slider", min:1, max:128}
NUM_WORKERS = 2 # @param {type: "slider", min:1, max:16}

EPOCHS = 1 # @param {type: "slider", min:1, max:128}
ACCELERATOR = "auto" # @param ["auto", "gpu", "tpu", "cpu"]

In [133]:
class SBERTDataset(Dataset):
    def __init__(self, file_path: str):
        self._data = pd.read_csv(
            file_path, sep="\t", keep_default_na=False, quoting=csv.QUOTE_NONE
        )

        self._sbert = SentenceTransformer("all-mpnet-base-v2")

        self._types = self._get_encoded_types()
        self._scores = torch.tensor(self._data["y_score"]).float()

    def _get_encoded_types(self):
        types_as_int = types_to_int(self._data["y_type"].tolist())
        encoded_types = torch.nn.functional.one_hot(
            torch.tensor(types_as_int), num_classes=len(TYPES_MAP)
        ).float()
        return encoded_types

    def __getitem__(self, index):
        x1 = self._data["x1"]
        x2 = self._data["x2"]

        a = self._sbert.encode(x1[index])
        b = self._sbert.encode(x2[index])
        c = np.concatenate((a, b))

        x = torch.tensor(c)
        y = (self._types[index], self._scores[index])
        return x, y

    def __len__(self):
        return self._types.shape[0]

In [134]:

TRAIN_VAL_SPLIT = 0.8


class SBERTDataModule(LightningDataModule):
    def __init__(
        self,
        train_path: str,
        test_path: str,
        batch_size: int,
        train_batch_size: int,
        num_workers: int,
    ):
        super().__init__()

        self._train_dataset = None
        self._val_dataset = None
        self._test_dataset = None

        self._batch_size = batch_size
        self._train_batch_size = train_batch_size
        self._num_workers = num_workers
        self._train_path = train_path
        self._test_path = test_path
        self._preparte_data_per_node = True

    def _split(self, dataset, proportion):
        a = int(len(dataset) * proportion)
        b = len(dataset) - a
        return random_split(dataset, (a, b))

    def prepare_data(self):
        pass

    def setup(self, stage):
        if self._train_dataset is not None:
            return

        self._train_dataset, self._val_dataset = self._split(
            SBERTDataset(self._train_path), TRAIN_VAL_SPLIT
        )
        self._test_dataset = SBERTDataset(self._test_path)

    def train_dataloader(self):
        return DataLoader(
            self._train_dataset,
            batch_size=self._train_batch_size,
            num_workers=self._num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self._val_dataset,
            batch_size=self._batch_size,
            num_workers=self._num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self._test_dataset,
            batch_size=self._batch_size,
            num_workers=self._num_workers,
        )

    def predict_dataloader(self):
        return DataLoader(
            self._test_dataset,
            batch_size=self._batch_size,
            num_workers=self._num_workers,
            shuffle=False,
        )


In [135]:
class SBERT_iSTS_Model(LightningModule):
    def __init__(
        self, sbert_model: str = "all-mpnet-base-v2", learning_rate: float = 0.001
    ):
        super().__init__()
        # kolasdam(TODO) sparametryzować
        self._scoring_head = torch.nn.Linear(in_features=768 * 2, out_features=1)
        self._class_head = torch.nn.Linear(
            in_features=768 * 2, out_features=len(TYPES_MAP)
        )
        self._learning_rate = learning_rate
        self.save_hyperparameters()

    def _step(self, batch, batch_idx, id: str):
        x, y = batch
        y_hat = self.forward(x)
        return self.loss(y, y_hat, id)

    def forward(self, x):
        score = torch.reshape(self._scoring_head(x), (-1,))
        cls = torch.nn.functional.softmax(self._class_head(x), dim=1)

        return cls, score

    def loss(self, y, y_hat, id):
        # Klasa i ocena uczone razem
        scoring_loss = torch.nn.functional.mse_loss(y_hat[1], y[1])
        class_loss = torch.nn.functional.binary_cross_entropy_with_logits(
            y_hat[0], y[0]
        )
        return scoring_loss + class_loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch, batch_idx, "train")
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch, batch_idx, "val")
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._step(batch, batch_idx, "test")
        return loss

    def predict_step(self, batch, batch_idx):
        x, y = batch
        types, scores = self.forward(x)

        return torch.argmax(types, dim=1), torch.clamp(
            torch.round(scores).int(), min=0, max=5
        )

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self._learning_rate)


In [136]:
model = SBERT_iSTS_Model()

In [137]:
data = SBERTDataModule(
    f'{DATA_DIR}/train.tsv', 
    f'{DATA_DIR}/test.tsv', 
    batch_size=BATCH_SIZE, 
    train_batch_size=TRAIN_BATCH_SIZE,
    num_workers=NUM_WORKERS
)

In [138]:
trainer = Trainer(accelerator=ACCELERATOR, max_epochs=EPOCHS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [139]:
trainer.fit(model, data)


  | Name          | Type   | Params
-----------------------------------------
0 | _scoring_head | Linear | 1.5 K 
1 | _class_head   | Linear | 12.3 K
-----------------------------------------
13.8 K    Trainable params
0         Non-trainable params
13.8 K    Total params
0.055     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/damiankolaska/Desktop/NLP_SBERT_interpretable_semantic_text_similarity/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
                                                                            

/home/damiankolaska/Desktop/NLP_SBERT_interpretable_semantic_text_similarity/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/249 [00:00<?, ?it/s] huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 0:  80%|███████▉  | 199/249 [04:22<01:05,  1.32s/it, loss=4.5, v_num=16] huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before th

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 249/249 [05:29<00:00,  1.32s/it, loss=4.5, v_num=16]


In [140]:
trainer.test(model, data)

/home/damiankolaska/Desktop/NLP_SBERT_interpretable_semantic_text_similarity/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Testing DataLoader 0: 100%|██████████| 128/128 [02:38<00:00,  1.24s/it]


[{}]

In [141]:
predictions = trainer.predict(model, data)

/home/damiankolaska/Desktop/NLP_SBERT_interpretable_semantic_text_similarity/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 199it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 128/128 [02:50<00:00,  1.33s/it]


In [143]:
fields_sep = ' // '

def preds_to_wa(wa_content: str, preds_lines):
    wa_lines = wa_content.splitlines()

    idx = 0
    result = []

    for line in wa_lines:
        line_res = line

        if '<==>' in line:
            fields = line.split(fields_sep)
            preds_fields = preds_lines[idx].split()

            fields[1] = preds_fields[1]
            fields[2] = preds_fields[2]

            line_res = fields_sep.join(fields)
            idx += 1

        result.append(line_res)
    
    return '\n'.join(result)

In [144]:
def flatten(t):
    return [item for sublist in t for item in sublist]

types_inv_map = {v: k for k, v in TYPES_MAP.items()}

types = list(map(lambda t: types_inv_map[t], flatten([t.tolist() for t, s in predictions])))
scores = flatten([s.tolist() for t, s in predictions])

predictions = [
    f"{index}\t{item[0]} {item[1]}\n" for index, item in enumerate(zip(types, scores))
]

In [145]:
wa_file = os.path.join(DATA_DIR, f"STSint.testinput.{DATASET}.wa")
wa_output_file = os.path.join(DATA_DIR, f"STSint.testinput.{DATASET}-predictions.wa")

with open(wa_file) as file:
    wa_test = file.read()

wa_predictions = preds_to_wa(wa_test, predictions)

with open(wa_output_file, "w") as file:
    file.write(wa_predictions)

In [146]:
from subprocess import check_output

cmds = [
    f"perl evalF1_penalty.pl {wa_file} {wa_output_file}",
    f"perl evalF1_no_penalty.pl {wa_file} {wa_output_file}",
]

for cmd in cmds:
    print(f"Executing {cmd}")
    print(check_output(cmd.split(), cwd="./").decode())

Executing perl evalF1_penalty.pl ./data/sem_eval_2016/headlines/STSint.testinput.headlines.wa ./data/sem_eval_2016/headlines/STSint.testinput.headlines-predictions.wa
Epoch 1:  35%|███▌      | 88/249 [16:00<29:16, 10.91s/it, loss=3.61, v_num=15]


FileNotFoundError: [Errno 2] No such file or directory: '/content/Mazury'